In [335]:
import numpy as np
import matplotlib.pyplot as plt

In [336]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
# input_str = clean_text_for_pattern_letter(template_str, out_len=500)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
111748
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [337]:
import copy

def divide_via_trace(inputs: list, decrease_rate: float = 0.5, increase_rate: float = 0.5):
    K: dict[str, int] = {}
    T = []
    
    prev_K: dict[str, int] = {}
    for inp_idx, inp in enumerate(inputs):
        # decrease
        for symbol, stats in K.items():
            # K[symbol] -= stats * decrease_rate
            # if stats > 0:
            #     K[symbol] -= 1 / (stats * decrease_rate)
            # else:
            #     K[symbol] = 0
            K[symbol] -= pow(decrease_rate, stats)

            if K[symbol] < 0:
                K[symbol] = 0

        # increase
        K.setdefault(inp, 0)
        K[inp] += pow(increase_rate, K[inp])
        # K[inp] += increase_rate

        # norm
        # K_D = {}
        # sum_K = sum(K.values())
        # for symbol, stats in K.items():
        #     K_D.setdefault(symbol, stats / sum_K)

        # compare to prev
        diff_sum = 0
        for symbol, stats in K.items():
            prev_stats = prev_K.get(symbol, 0)

            diff = abs(stats - prev_stats)
            diff_sum += diff

        for symbol, stats in prev_K.items():
            if not K.get(symbol):
                diff_sum += stats

        # get sum
        S = []
        for symbol, stats in K.items():
            S.append(stats)
        
        S = np.array(S, dtype='float')
        
        # t = np.mean(S)
        # t = np.sum(S)
        t = diff_sum
        T.append(t)

        # recur
        if inp_idx > 0:
            prev_K = copy.deepcopy(K)
    
    return T


In [338]:
def get_substrings(T:list, input:list):
    substrings: dict[str, int] = {}
    buffer = []

    for inp_idx, inp in enumerate(input):
        buffer.append(inp)

        if inp_idx == 0 or inp_idx >= len(input) - 3:
            continue

        if T[inp_idx] < T[inp_idx - 1] and T[inp_idx] < T[inp_idx + 1] and T[inp_idx + 3] > T[inp_idx - 1]:
            substr = ''.join(buffer)

            substrings.setdefault(substr, 0)
            substrings[substr] += 1

            buffer = []

    # sort
    substrings = dict(sorted(substrings.items(), key=lambda item: item[1], reverse=True))

    return substrings


In [339]:
def plot_T(T:list):
    T_arr = np.array(T, dtype=float)

    fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
    axs.plot(np.arange(T_arr.size), T_arr)

    plt.show()

In [340]:
def plot_T_with_symbols(T:list, inputs: list):
    T_arr = np.array(T, dtype=float)

    fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
    axs.plot(np.arange(T_arr.size), T_arr)
    axs.xaxis.set_ticks(np.arange(len(inputs)), inputs)
    axs.xaxis.grid(True)

    plt.show()

In [341]:
T = divide_via_trace(inputs=input_str, decrease_rate=0.5, increase_rate=0.5)
# plot_T_with_symbols(T=T, inputs=input_str)
# plot_T(T=T)
substrs = get_substrings(T, input=input_str)

print(f'{len(substrs)=}')
len_arr = []
for substr, stats in substrs.items():
    print(substr, stats)
    len_arr.append(len(substr))

len_arr = np.array(len_arr, dtype='float')
print(f'{np.median(len_arr)=}')
print(f'{np.sum(len_arr)=} =? {len(input_str)}')

len(substrs)=2500
ie the poo 40
h loo 14
leston cott 11
 said poo 10
 rabb 7
ie the pooh h 5
d afternoo 5
it and all 4
le idd 4
yore gloo 4
h rubb 4
yore said poo 4
 tell 4
 right roo 4
n balloo 3
 with bee 3
le tidd 3
ed and pull 3
 pull 3
 of a sudd 3
h poo 3
h nodded 3
h and poo 3
ing a litt 3
h goo 3
le and i reply cott 3
o poo 3
h said rabb 3
 three 3
o roo 3
le fell 3
 o 3
 call 3
k at me swimm 3
h sitt 3
a said poo 3
ie ther poo 2
 buzz 2
ked round to see 2
h well 2
n bee 2
h i shall 2
p sigh picked his bear up by the leg and walked off 2
 to the doo 2
h behind him at the doo 2
 and went out and in a moment i heard winn 2
ed out poo 2
o rabb 2
it loo 2
it too 2
h nodd 2
shall 2
h and rabb 2
ch tree 2
d to loo 2
h stopp 2
identall 2
 be all 2
yore stoo 2
h i'm sorr 2
o pooh h 2
ad bee 2
ked at the bell 2
 the very deep p 2
it until he was half way down when it would be too 2
y writt 2
 piglet trott 2
d morning ee 2
d morning poo 2
h that would not be a goo 2
y birthday with love 

In [342]:
# plot_T(T)

In [343]:
# for dr in range(1, 10):
#     decrease_rate = dr * 0.1
#     print(decrease_rate)
#     T = divide_via_trace(inputs=input_str, decrease_rate=decrease_rate)
#     plot_T(T)